# Preparação e Precessamento dos dados 


## Conferindo a integridade básica dos dados

Na etapa anterior, [Coleta e Importação dos Dados]("./03_coleta_e_importação_dos_dados.ipynb"), utilizamos o seguinte script para importar os dados e transformar o nome dos arquivos em um padrão adequado: 

In [1]:
import os
import re
import shutil
import pandas as pd

In [2]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith(".xls"):
            match = pattern.search(filename)
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename)

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_15545/4186165578.py:44: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_15545/4186165578.py:44: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_15545/4186165578.py:44: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


Observe que, ao realizar a importação, recebemos um aviso `DtypeWarning`. Iremos começar verificar a consistência e integridade dos dados compreendendo a razão pela qual este aviso ocorre e checar como podemos processar os dados adequadamente para torná-los úteis para análise. 

### Solucionando os problemas de tipo dos dados

A saída `DtypeWarning` do comando de importação nos avisa que o `pandas` não conseguiu identificar o tipo de algumas colunas do dataset e, por isso, teve que inferir um tipo. Em outras palavras, o `pandas` tenta determinar os tipos de cada coluna; uma operação que exige muita memória.  

Podemos silenciar esses avisos basicamente de duas formas: A primeira é utilizando o parâmetro `dtypes` e passando `object` como valor.`Object` [é a classe base de onde derivam toda as classes em Python](https://docs.python.org/pt-br/3/library/functions.html?highlight=object#object), ou seja, `object` pode conter qualquer objeto Python. Isso fará com que o `pandas` defina o tipo `object` para todos os dados. Esse tipo já é aplicado para tipos de dados mistos ou que o `pandas` não consegue inferir. [No entanto, essa medida deve ser evitado na medida do possível](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes) (para desempenho e interoperabilidade com outras bibliotecas e métodos).

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                dtypes=object) 
                for file in file_path_genarator]
```

A outra forma é usar o parâmetro `low-memory=False`, como sugere o próprio aviso: `DtypeWarning: Columns (...,...) have mixed types. Specify dtype option on import or set low_memory=False`.

```python
...

datasetes = [pd.read_csv(file,
                sep="\t",
                enconding="UTF-16 LE",
                low-memory=True) 
                for file in file_path_genarator]
```
 Porém, esse parâmetro, embora aparente funcionar, é considerado obsoleto e pode levar a erros silenciosos que não são documentados. [Existe uma discussão que se arrasta desde 2014 sobre se este parâmetro deve ou não ser melhor documentado ou retirado da biblioteca](https://github.com/pandas-dev/pandas/issues/5888). O fato é que sua utilização não é muito comum apesar do aviso indicar seu uso.

De toda forma, nenhuma das abordagens resolvem o problema de fato. O `pandas` continuará usando muito recurso para adivinhar os tipos. 

Portanto, a abordagem mais indicada nesses casos é fornecer os tipos manualmente. 

Antes de partir para esta tarefa, vamos checar o estado do DataFrame:


In [2]:
df.shape

(119158, 54)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o


A saída do método `info()` nos fornece as seguintes informações:

**Column**: Esta coluna indica os nomes das colunas presentes no DataFrame.

**Non-Null Count**: Esta coluna mostra a contagem de valores não nulos em cada coluna. Em outras palavras, é o número de entradas que possuem valores válidos para aquela coluna. Por exemplo, na primeira coluna "ANO_BO", existem 119158 valores não nulos.

**Dtype**: Esta coluna informa o tipo de dados (data type) presente em cada coluna. Por exemplo, `object`, como vimos, indica que os registros podem qualquer objeto python, geralmente são strings ou valores mistos. Manter os dados assim pode causar inúmeros problemas já que strings e não-strings podem estar juntas em uma mesma série. Trabalhar com os tipos corretos no DataFrame é extremamente importante para que possamos garantir:

- Precisão nos dados
- Economia de memória
- Desempenho
- Manipulação dos dados mais eficaz
- Prevenção de erros
- Consistência. 

Podemos notar que, com exceção de alguns tipos `float64`, o pandas inferiu `object` em todo o conjunto de dados. Vamos olhar uma amostra aleatória dos registros para termos uma noção da tipagem que precisará ser adequada.

In [5]:
df.sample(10)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
34734,2023,121,121/2023,31/05/2023 14:41:33,31/05/2023 14:41:33,30/05/2023,02:00,DE MADRUGADA,30/05/2023,31/05/2023 14:41:33,Conhecida,Sim,120/2023 - 40201,AVENIDA PRINCIPAL,916.0,AREA RURAL,CUBATAO,SP,"-23,875137418","-46,442616248",Escritório,NaN,BO PARA FLAGRANTE,1ª DIG - DEIC - DEINTER 6,01º D.P. CUBATÃO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Outros
24613,2023,2898,2898/2023,17/05/2023 12:22:37,17/05/2023 12:22:37,16/05/2023,NaN,A NOITE,17/05/2023,17/05/2023 12:22:37,Desconhecida,Não,NaN,Rua Chico Mana,200.0,Jardim Maria Duarte,S.PAULO,SP,"-23,6187579","-46,7521173",Via pública,NaN,BO PARA REGISTRO,37º D.P. CAMPO LIMPO,37º D.P. CAMPO LIMPO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
66566,2023,1173,1173/2023,20/02/2023 14:09:21,20/02/2023 14:09:21,18/02/2023,19:00,A NOITE,20/02/2023,20/02/2023 14:09:21,Desconhecida,Não,NaN,RUA GUAIANASES,1.0,SANTA CECILIA,S.PAULO,SP,"-23,537025393","-46,643326827",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,08º D.P. BRAS,03º D.P. CAMPOS ELISEOS,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - TRANSEUNTE,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Samsung
65027,2023,263697,263697/2023,18/02/2023 09:20:41,18/02/2023 09:20:46,16/02/2023,11:30,PELA MANHÃ,16/02/2023,18/02/2023 09:20:41,Desconhecida,Não,NaN,RUA CORONEL FRANCISCO AMARO,2.0,BRAS,S.PAULO,SP,"-23,5404291049999","-46,6205841789999",Via pública,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,12º D.P. PARI,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - OUTROS,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
40131,2023,3715,3715/2023,08/06/2023 12:35:01,08/06/2023 12:35:01,01/06/2023,07:00,PELA MANHÃ,01/06/2023,08/06/2023 12:35:01,Desconhecida,Não,3673/2023 - 10108,ACESSO PARA RODOVIA AYRTON SENNA,0.0,SAO MIGUEL,S.PAULO,SP,"-23,4847952969999","-46,4529307",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,08º D.P. BRAS,63º D.P. VILA JACUI,Localização e/ou Devolução,Localização/Apreensão de veículo,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BWO2A51,SP,S.PAULO,Azul,JEEP/COMMANDER OVR TD380,2022.0,2022.0,UTILITÁRIO,1.0,Apple
20120,2023,3577,3577/2023,10/05/2023 19:38:58,10/05/2023 19:38:58,06/05/2023,NaN,DE MADRUGADA,10/05/2023,10/05/2023 19:38:58,Desconhecida,Não,NaN,Rua Barão do Rio Branco,555.0,Santo Amaro,S.PAULO,SP,"-23,6527913","-46,7085313",Via pública,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,DEL.POL.PLANTÃO ATIBAIA,11º D.P. SANTO AMARO,Título II - Patrimônio (arts. 155 a 183),Roubo (art. 157) - INTERIOR DE VEICULO,NaN,Consumado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
80362,2023,811,811/2023,12/04/2023 05:02:36,12/04/2023 05:02:36,11/04/2023,19:00,A NOITE,11/04/2023,12/04/2023 05:02:36,Desconhecida,Não,NaN,NaN,0.0,JARDIM AMANDA,HORTOLANDIA,SP,NaN,NaN,Residência,NaN,APRECIAÇÃO DO DELEGADO TITULAR,03º D.P. AMERICANA,02º D.P. HORTOLÂNDIA,Título II - Patrimônio (arts. 155 a 

Ao observar os registros do DataFrame, podemos notar o seguinte:

1. Existem inconsistências nos registros: Além das datas estarem em formato inadequado para a maioria dos softwares e fora do padrão para dados,  alguns deles estão com caracteres maiúsculos e outros minúsculos. Será necessário padronizar os tipos de data e as strings e limpar os espaços extras. 
2. Os registros de `LATITUDE` e `LONGITUDE` possuem um espaço extra tanto à esquerda quanto à direita. Será necessário limpar estes espaços, substituir a "," por "." e converter o dados adequadamente para `float`.
3. A única coluna que se refere a valores do tipo `int` e que pode ser utilizada para cálculo é "QUANT_CELULAR", que se refere a quantidade de dispositivos roubados da vítima. Esta variável precisa ser convertida para o tipo adequado que suporte valores ausentes.
4. Existem muitos valores ausentes. Será necessário pensar em uma abordagem para lidar com eles.

Antes de partir para solução desses problemas, prefiro examinar as informações do dataset fornecidas pela Secretaria de Segurança. 


#### Conferindo o dicionário de dados 

Como uma tarefa complementar desta etapa, para facilitar a comparação entre o dicionário e o DataFrame, eu exportei a saída da propriedade `dtypes` e copiei a tabela do documento "METODOLOGIA" para uma planilha a fim de:

1. Checar se o DataFrame contém todas as variáveis presentes no dicionário de dados;
2. Identificar os tipos dos dados e atribuí-los adequadamente ao DataFrame.

A planilha se encontra no diretório `data` e, como poderá ser observado, fiz uma busca vertical (VLOOKUP ou PROCV) para cruzar as colunas que se referem às características do conjunto de dados. O código da exportação dos dtypes segue abaixo:

In [8]:
t = df.dtypes.reset_index()
t.columns = ["Variáveis", "Tipos"]
t.to_excel("./data/dtypes_out.xlsx", index=False)

Ao analisar o dicionário de dados, cheguei à conclusão de que ele oferece pouca ajuda. Os motivos são destacados a seguir:

1. O dicionário não informa os tipos dos dados;
2. As descrições das variáveis são vagas e não fornecem mais informações além do óbvio;
3. Os rótulos das variáveis especificadas no dicionário são diferentes dos rótulos do DataFrame, mas algumas se referem à mesma coisa. Por exemplo: `BOLETIM_EMITIDO`, uma das variáveis do DataFrame, não consta no dicionário, mas é equivalente à "DATAHORA_IMPRESSAO_BO" definida apenas como "Data-hora da elaboração da impressão" no dicionário.
4. O DataFrame possui 54 variáveis enquanto o dicionário indica 62. Dada a condição anterior, não consigo afirmar com certeza se o DataFrame possui variáveis não especificadas no dicionário de dados ou se as variáveis do dicionário se referem exclusivamente às do DataFrame.
5. O dicionário de dados informa que, várias linhas podem se referir ao mesmo boletim. Portanto, segundo o documento, para conclusões quanto as quantidades de ocorrências é necessária exclusão das duplicidades das seguintes variáveis:

- NOME_DELEGACIA
- ANO_BO
- NUM_BO

Isso será checado em detalhes posteriormente.


#### Corrigindo os tipos e limpando os dados

Para formatar corretamente os registros de Longitude e Latitude usaremos a seguinte função:

In [3]:
"""Limpa os registro LONGITUDE e LATITUDE """

def clean_coordinate_value(value):
    if isinstance(value, str):
        try:
            clean_value = value.strip().replace(",", ".")
            return float(clean_value)
        except ValueError:
            return None 
        
    return None

Para fornecer os tipos corretos dos registros, irei realizar uma nova importação utilizando os argumentos nomeados definidos no método `read_csv()`.

O método `read_csv()` possui diversos argumentos nomeados que podemos fornecer para cuidar da formatação dos registros. Estes argumentos precisam ser estruturas de dados específicas:
- `dtype`: Precisa ser um tipo, como `str`, ou um dicionário. Ele define os tipos de dados a serem aplicados a todo o conjunto de dados ou a colunas individuais
- `parse_dates`: `bool`, lista de Hashable, lista de listas ou dict de {Hashable list}, padrão `False`.  
- `date_format`: `str` ou `dict`  com nomes das colunas e um formato opcional
- `converters`: `dict` de {Hashable Callable}, optional. Aqui tem um peculiaridade: Se `converters` for definido, ele terá prioridade em vez do `dtype`.

Para passarmos essas estruturas para o método `read_csv()`, precisamos definilas:


In [4]:
DTYPES = {
    "ANO_BO": pd.StringDtype(),
    "ANO_FABRICACAO": pd.StringDtype(),
    "ANO_MODELO": pd.StringDtype(),
    "BO_AUTORIA": pd.StringDtype(),
    "CIDADE": pd.StringDtype(),
    "BAIRRO": pd.StringDtype(),
    "LOGRADOURO": pd.StringDtype(),
    "CIDADE_VEICULO": pd.StringDtype(),
    "CORCUTIS": None,
    "DELEGACIA_CIRCUNSCRICAO": pd.StringDtype(),
    "DELEGACIA_NOME": pd.StringDtype(),
    "DESCR_COR_VEICULO": pd.StringDtype(),
    "DESCR_MARCA_VEICULO": pd.StringDtype(),
    "DESCR_TIPO_VEICULO": pd.StringDtype(),
    "DESCRICAOLOCAL": pd.StringDtype(),
    "DESDOBRAMENTO": pd.StringDtype(),
    "ESPECIE": pd.StringDtype(),
    "ESTADOCIVIL": pd.StringDtype(),
    "EXAME": None,
    "FLAGRANTE": pd.StringDtype(),
    "GRAUINSTRUCAO": pd.StringDtype(),
    "IDADE": pd.Int64Dtype(),
    "MARCA_CELULAR": pd.StringDtype(),
    "NACIONALIDADE": pd.StringDtype(),
    "NATURALIDADE": pd.StringDtype(),
    "NATUREZAVINCULADA": pd.StringDtype(),
    "NUM_BO": pd.StringDtype(),
    "NUMERO": pd.StringDtype(),
    "NUMERO_BOLETIM": pd.StringDtype(),
    "NUMERO_BOLETIM_PRINCIPAL": pd.StringDtype(),
    "PARENTESCO": pd.StringDtype(),
    "PERIDOOCORRENCIA": pd.StringDtype(),
    "PLACA_VEICULO": pd.StringDtype(),
    "PROFISSAO": pd.StringDtype(),
    "QUANT_CELULAR": pd.Int64Dtype(),
    "RELACIONAMENTO": pd.StringDtype(),
    "RUBRICA": pd.StringDtype(),
    "SEXO": pd.StringDtype(),
    "SOLUCAO": pd.StringDtype(),
    "STATUS": pd.StringDtype(),
    "TIPOPESSOA": pd.StringDtype(),
    "TIPOVINCULO": pd.StringDtype(),
    "UF": pd.StringDtype(),
    "UF_VEICULO": pd.StringDtype(),
    "VITIMAFATAL": pd.StringDtype(),
}

DATE_TYPES = [
    "BO_EMITIDO",
    "BO_INICIADO",
    "DATACOMUNICACAO",
    "DATAELABORACAO",
    "DATANASCIMENTO",
    "DATAOCORRENCIA",
]

DATE_FORMAT = {
    "BO_EMITIDO": "%d/%m/%Y %H:%M:%S",
    "BO_INICIADO": "%d/%m/%Y %H:%M:%S",
    "DATACOMUNICACAO": "%d/%m/%Y",
    "DATAELABORACAO": "%d/%m/%Y %H:%M:%S",
    "DATANASCIMENTO": "%d/%m/%Y",
    "DATAOCORRENCIA": "%d/%m/%Y",
    "HORAOCORRENCIA": "%H:%M",
}

CONVERTERS = {
    "LATITUDE": clean_coordinate_value,
    "LONGITUDE": clean_coordinate_value,
}

TO_UPPER_LIST = [
    "ANO_BO",
    "ANO_FABRICACAO",
    "ANO_MODELO",
    "BO_AUTORIA",
    "CIDADE",
    "BAIRRO",
    "LOGRADOURO",
    "CIDADE_VEICULO",
    "DELEGACIA_CIRCUNSCRICAO",
    "DELEGACIA_NOME",
    "DESCR_COR_VEICULO",
    "DESCR_MARCA_VEICULO",
    "DESCR_TIPO_VEICULO",
    "DESCRICAOLOCAL",
    "DESDOBRAMENTO",
    "ESPECIE",
    "ESTADOCIVIL",
    "FLAGRANTE",
    "GRAUINSTRUCAO",
    "MARCA_CELULAR",
    "NACIONALIDADE",
    "NATURALIDADE",
    "NATUREZAVINCULADA",
    "NUM_BO",
    "NUMERO",
    "NUMERO_BOLETIM",
    "NUMERO_BOLETIM_PRINCIPAL",
    "PARENTESCO",
    "PERIDOOCORRENCIA",
    "PLACA_VEICULO",
    "PROFISSAO",
    "RELACIONAMENTO",
    "RUBRICA",
    "SEXO",
    "SOLUCAO",
    "STATUS",
    "TIPOPESSOA",
    "TIPOVINCULO",
    "UF",
    "UF_VEICULO",
    "VITIMAFATAL"
]


Poderíamos passar `str` para os campos do dict `DTYPES` referentes a strings, contudo, ainda teríamos registros avaliados como `object`. Portanto, a [maneira indicada de trabalharmos com registros de texto no `pandas` é utilizando a classe `StringDtype()`](https://pandas.pydata.org/docs/user_guide/text.html#string-methods). 

Outro ponto importante de destacar é a utilização da classe `Int64Dtype()` utilizada para a variável "QUANT_CELULAR". Usar `int`, neste caso, resultaria na exceção `ValueError: invalid literal for int() with base 10` que indica que o `pandas` não consegue converter uma string em um número. Essa confusão acontece, pois o `pandas` interpretou a variável como `object` para poder suportar valores nulos. Como a função embutida `int` é intolerante e levanta uma exceção quando esbarra com um valor que não consegue converter (como valores nulos), precisamo usar `Int64Dtype()` que é um tipo de numeral inteiro anulável. 

Por fim, a constante `TO_UPPER_LIST` será utilizada para transformar os caracteres minúsculos em maiúsculo sem que o tipo da `Serie` volte a ser `object`.

In [5]:
"""
Padroniza os nomes dos arquivos para "DadosBO_SP_{ano}_{mes}.xls
"""
def process_file(input_dir, output_dir):

    PATTERN = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")

    MONTHS = {
        "1": "Janeiro",
        "2": "Fevereiro",
        "3": "Março",
        "4": "Abril",
        "5": "Maio",
        "6": "Junho",
        "7": "Julho",
        "8": "Agosto",
        "9": "Setembro",
        "10": "Outubro",
        "11": "Novembro",
        "12": "Dezembro"
    }

    for filename in os.listdir(input_dir):
        # Verificar se o arquivo é um arquivo xls
        if filename.endswith(".xls"):
            # Pesquisar o padrão
            match = PATTERN.search(filename)
            # Se verdadeiro, capturar o dígito da string e utilizá-lo para obter o valor da chave do dict `meses`
            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                # Novo nome do arquivo após substituição
                new_name = pattern.sub(f"{name_month}", filename)

                # Caminhos completos dos arquivos antigo e novo
                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path

In [6]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

file_path_generator = process_file(input_dir, output_dir)

# Passa as estruturas de dados pra read_csv realizar as operações na importação:
datasets = [pd.read_csv(file, sep='\t',
                        encoding="UTF-16 LE",
                        dtype=DTYPES,
                        parse_dates=DATE_TYPES,
                        date_format=DATE_FORMAT,
                        converters=CONVERTERS)
            for file in file_path_generator]

df_temp = pd.concat(datasets, axis=0, ignore_index=True)


In [8]:
# https://pandas.pydata.org/docs/user_guide/text.html#string-methods 
# Transforma os caracteres em maiúsculo
for c in TO_UPPER_LIST:
    df_temp[c] = df_temp[c].str.upper()
    df_temp.columns.str.strip()


pd.set_option("display.max_columns", None)

print(df_temp.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   ANO_BO                    119158 non-null  string        
 1   NUM_BO                    119158 non-null  string        
 2   NUMERO_BOLETIM            119158 non-null  string        
 3   BO_INICIADO               119158 non-null  datetime64[ns]
 4   BO_EMITIDO                119158 non-null  datetime64[ns]
 5   DATAOCORRENCIA            119158 non-null  datetime64[ns]
 6   HORAOCORRENCIA            110412 non-null  object        
 7   PERIDOOCORRENCIA          119158 non-null  string        
 8   DATACOMUNICACAO           119158 non-null  datetime64[ns]
 9   DATAELABORACAO            119158 non-null  datetime64[ns]
 10  BO_AUTORIA                119158 non-null  string        
 11  FLAGRANTE                 119158 non-null  string        
 12  NU

In [9]:
df_temp.sample(5)

,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,BO_AUTORIA,FLAGRANTE,NUMERO_BOLETIM_PRINCIPAL,LOGRADOURO,NUMERO,BAIRRO,CIDADE,UF,LATITUDE,LONGITUDE,DESCRICAOLOCAL,EXAME,SOLUCAO,DELEGACIA_NOME,DELEGACIA_CIRCUNSCRICAO,ESPECIE,RUBRICA,DESDOBRAMENTO,STATUS,TIPOPESSOA,VITIMAFATAL,NATURALIDADE,NACIONALIDADE,SEXO,DATANASCIMENTO,IDADE,ESTADOCIVIL,PROFISSAO,GRAUINSTRUCAO,CORCUTIS,NATUREZAVINCULADA,TIPOVINCULO,RELACIONAMENTO,PARENTESCO,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
114231,2023,470212,470212/2023,2023-03-26 08:00:56,2023-03-26 08:01:00,2023-03-25,13:30,A TARDE,2023-03-25,2023-03-26 08:00:56,DESCONHECIDA,NÃO,<NA>,RUA FREI GASPAR,1,CIDADE NAÚTICA,S.VICENTE,SP,NaN,NaN,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,02º D.P. SÃO VICENTE,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,MOTOROLA
94759,2023,869,869/2023,2023-03-01 17:51:31,2023-03-01 17:51:31,2023-02-28,10:30,PELA MANHÃ,2023-03-01,2023-03-01 17:51:31,DESCONHECIDA,NÃO,<NA>,RUA ALCÂNTARA MACHADO,180,VILA VARELA,POA,SP,-23.512865,-46.335699,VIA PÚBLICA,NaN,ENCAMINHAMENTO DP ÁREA DO FATO,04º D.P. GUARULHOS,DEL.POL.POA,LOCALIZAÇÃO E/OU DEVOLUÇÃO,LOCALIZAÇÃO/APREENSÃO E ENTREGA DE VEÍCULO,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,SFP5F71,ES,VILA VELHA,PRETA,SR/RANDON SR CA 3E,2022,2022,SEMI-REBOQUE,1,SAMSUNG
41958,2023,907783,907783/2023,2023-06-12 00:36:29,2023-06-12 00:36:35,2023-06-11,21:00,A NOITE,2023-06-11,2023-06-12 00:36:29,DESCONHECIDA,NÃO,<NA>,AVENIDA NOVE DE JULHO,3,JARDIM PAULISTA,S.PAULO,SP,-23.548164,-46.638753,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,03º D.P. CAMPOS ELISEOS,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,SAMSUNG
37190,2023,868068,868068/2023,2023-06-04 00:31:18,2023-06-04 00:31:23,2023-06-03,17:45,A TARDE,2023-06-03,2023-06-04 00:31:18,DESCONHECIDA,NÃO,<NA>,RUA CALIFORNIA,994,ITAIM BIBI,S.PAULO,SP,-23.612953,-46.685358,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,96º D.P. MONÇÕES,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE
32649,2023,829314,829314/2023,2023-05-28 21:50:58,2023-05-28 21:51:05,2023-05-28,20:00,A NOITE,2023-05-28,2023-05-28 21:50:58,DESCONHECIDA,NÃO,<NA>,AVENIDA ELLIS MAAS,257,CAPAO REDONDO,S.PAULO,SP,-23.660759,-46.768024,VIA PÚBLICA,NaN,BO PARA REGISTRO,DELEGACIA ELETRONICA,47º D.P. CAPAO REDONDO,TÍTULO II - PATRIMÔNIO (ARTS. 155 A 183),ROUBO (ART. 157) - OUTROS,<NA>,CONSUMADO,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,1,APPLE


---

## Referencias

- [Link da documentação para trabalhar a consistencia das datas](https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html)